In [ ]:

!pip install faiss-cpu sentence-transformers



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:

import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [ ]:
# Sample health advice dataset
data = {
    "Category": ["Nutrition", "Fitness", "Mental Health", "Sleep", "General"],
    "Question": [
        "What are the benefits of eating fruits daily?",
        "How often should I exercise in a week?",
        "What are signs of stress or anxiety?",
        "How many hours of sleep do adults need?",
        "How can I boost my immune system?"
    ],
    "Answer": [
        "Fruits provide essential vitamins, fiber, and antioxidants. They help improve digestion and reduce the risk of chronic diseases.",
        "Aim for at least 150 minutes of moderate aerobic activity or 75 minutes of vigorous activity each week, along with strength training.",
        "Common signs include constant worry, restlessness, fatigue, and difficulty concentrating or sleeping.",
        "Most adults need 7 to 9 hours of sleep per night to maintain good health and optimal brain function.",
        "Stay hydrated, eat a balanced diet rich in vegetables and fruits, get enough sleep, and manage stress levels."
    ]
}
df = pd.DataFrame(data)
df.head()


,Category,Question,Answer
0,Nutrition,What are the benefits of eating fruits daily?,"Fruits provide essential vitamins, fiber, and ..."
1,Fitness,How often should I exercise in a week?,Aim for at least 150 minutes of moderate aerob...
2,Mental Health,What are signs of stress or anxiety?,"Common signs include constant worry, restlessn..."
3,Sleep,How many hours of sleep do adults need?,Most adults need 7 to 9 hours of sleep per nig...
4,General,How can I boost my immune system?,"Stay hydrated, eat a balanced diet rich in veg..."


In [ ]:
# Encode questions with SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
question_embeddings = model.encode(df["Question"].tolist(), convert_to_numpy=True)

# Create FAISS index
dimension = question_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(question_embeddings)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Health advice query function
def get_health_advice(user_query, top_k=1):
    query_embedding = model.encode([user_query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k)
    for i in indices[0]:
        question = df.iloc[i]["Question"]
        answer = df.iloc[i]["Answer"]
        category = df.iloc[i]["Category"]
        return f"📚 *Category:* {category}\n❓ *Related Question:* {question}\n💡 *Advice:* {answer}"


In [ ]:
query = "How can I stay healthy and avoid getting sick?"
print(get_health_advice(query))


📚 *Category:* General
❓ *Related Question:* How can I boost my immune system?
💡 *Advice:* Stay hydrated, eat a balanced diet rich in vegetables and fruits, get enough sleep, and manage stress levels.


In [ ]:
print(get_health_advice("How much sleep do I need at night?"))


📚 *Category:* Sleep
❓ *Related Question:* How many hours of sleep do adults need?
💡 *Advice:* Most adults need 7 to 9 hours of sleep per night to maintain good health and optimal brain function.


In [ ]:
questions = [
    "How can I improve my mental health?",
    "Is walking good for fitness?",
    "What foods help boost immunity?",
    "Why is sleep important?"
]

for q in questions:
    print(f"🧑‍⚕️ Question: {q}")
    print(get_health_advice(q))
    print("-" * 50)


🧑‍⚕️ Question: How can I improve my mental health?
📚 *Category:* General
❓ *Related Question:* How can I boost my immune system?
💡 *Advice:* Stay hydrated, eat a balanced diet rich in vegetables and fruits, get enough sleep, and manage stress levels.
--------------------------------------------------
🧑‍⚕️ Question: Is walking good for fitness?
📚 *Category:* Fitness
❓ *Related Question:* How often should I exercise in a week?
💡 *Advice:* Aim for at least 150 minutes of moderate aerobic activity or 75 minutes of vigorous activity each week, along with strength training.
--------------------------------------------------
🧑‍⚕️ Question: What foods help boost immunity?
📚 *Category:* General
❓ *Related Question:* How can I boost my immune system?
💡 *Advice:* Stay hydrated, eat a balanced diet rich in vegetables and fruits, get enough sleep, and manage stress levels.
--------------------------------------------------
🧑‍⚕️ Question: Why is sleep important?
📚 *Category:* Sleep
❓ *Related Questi